In [ ]:
import os

import keras
from keras.layers import Input, Concatenate
from keras.models import Model, load_model
from tensorflow.keras.optimizers import legacy as legacy_optimizers

In [ ]:
model_dir = '../checkpts-tmp/'
model_names = [
    '48px_v0.4SSL4EO-MLPep21_2025-09-27.h5',
    '48px_v0.6SSL4EO-MLPep60_2025-10-17.h5',
    '48px_v0.8SSL4EO-MLPep28_2025-10-19.h5',
    '48px_v0.9SSL4EO-MLPep28_2025-10-19.h5',
    '48px_v1.0SSL4EO-MLPep36_2025-10-21.h5',
    '48px_v1.3SSL4EO-MLPep46_2025-10-21.h5'
]
    
models = []
for name in model_names:
    model = load_model(os.path.join(model_dir, name))
    model._name = name[:15]
    models.append(model)

In [ ]:
for model in models:
    print(model.summary())

In [ ]:
input_layer = Input(shape=models[0].input_shape[1:]) 

In [ ]:
for model in models:
    assert input_layer.shape == model.input_shape

In [ ]:
model_outputs = []
for model in models:  
    model_output = model(input_layer)
    model_outputs.append(model_output)

concatenated_output = Concatenate()(model_outputs)
ensemble_model = Model(input_layer, concatenated_output)

ensemble_model.compile(
        optimizer=legacy_optimizers.Adam(3e-4),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[keras.metrics.BinaryAccuracy(name="acc")],
    )
ensemble_model._name = '48px_v0.X-SSL4EO-MLPensemble_2025-10-21'
ensemble_model.summary()

In [ ]:
ensemble_model.save(os.path.join(model_dir, f'{ensemble_model._name}.h5'))